In [1]:
###Hide
import numpy as np
import pandas as pd
import scipy as sp
from sklearn import preprocessing
from sklearn.cross_validation import KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.ensemble import AdaBoostClassifier as AdaBoost
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn import grid_search
from sklearn.decomposition import PCA
from sklearn import feature_selection as fs
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

## Data Scrubbing Process

#### Summary of the data

The National Child Development Study (NCDS) was a perinated mortality survey which examined the social factors assosiated with stillbirth and infant mortality over 17000 babies born in the UK during 1958. Since then the cohort has been survery on six other times in order to monitor their health. These surveys were carried out in 1965 (age 7), 1969 (age 11), 1974 (age 16), 1981 (age 23), 1991 (age 33) and 1999/2000 (age 41/42). As part of the 1991 survey, a special study was also undertaken of the children of one third of the cohort members, including assessments of the behaviour and cognitive development of approximately 5000 children. There have also been surveys of sub-samples of the cohort, the most recent occurring in 1996 (age 37) when information was collected on the basic skills of a 10% sample of cohort members. [[reference]](../references/ncds_and_bcs70_response.pdf)

There are three files that contain the survey responses from the various subjects. Our analysis will be for these three files.

File Name : ncds0123
, Number of variables = 	1765
, Number of cases = 	18558


File Name : ncds_pms_additionals
, Number of variables = 	54
, Number of cases = 	16990


File Name : ncds_response
, Number of variables = 	18
, Number of cases = 	18558

In the next section we will follow the data science pipline process to explore and clean the data

## Explore and Clean Data

First task will be to read, explore, and clean the data.

#### Step 1: Load the data

In [2]:
#Load and inspect the ncds data
ncds_data = pd.read_csv('datasets/ncds0123.txt', delimiter='\t', low_memory=False)
# Print shapes
print "Shape of data:", ncds_data.shape
ncds_data.head()

Shape of data: (18558, 1765)


,ncdsid,n622,n0region,n1region,n2region,n3region,n553,n545,n520,n490,...,n1849,dvht07,dvht11,dvht16,dvrwt07,dvrwt11,dvrwt16,dvwt07,dvwt11,dvwt16
0,N10001N,2,9,9,9,9,23,4,2,12,...,-1,1.21899986267032,1.47299957275415,1.59999942779607,110.347991943347,98.1929931640604,105.055999755876,25.8549957275385,37.6489868164152,56.0199890136717
1,N10002P,1,9,8,8,8,34,4,5,1,...,-1,1.34599971771224,-1,-1,90.865997314449,-1,-1,26.3089904785155,-1,-1
2,N10003Q,1,4,4,4,4,34,4,10,1,...,-1,1.32099914550831,1.49899959564243,1.87999916076665,87.9599914550983,96.4049987792867,89.382995605487,24.4939880371087,38.1019897460905,66.6799926757659
3,N10004R,2,1,1,1,1,26,4,11,1,...,-1,1.29499912262003,1.51099967956562,1.62999916076665,105.16198730471,111.588989257796,132.054992675766,28.122985839843,45.8139953613169,72.7999877929584
4,N10005S,2,10,10,10,10,25,4,1,3,...,-1,1.34599971771224,-1,-1,129.382995605487,-1,-1,37.6489868164152,-1,-1


In [3]:
#Load and inspect the pms additions data
ncds_pms_data = pd.read_csv('datasets/ncds_pms_additionals.txt', delimiter='\t', low_memory=False)
# Print shapes
print "Shape of data:", ncds_pms_data.shape
ncds_pms_data.head()

Shape of data: (16990, 54)


,NCDSID,N622,BSTATUS,POD,BOOKING,PLANC,DIASTOL,MAXDBP,ALBECL,XRAY,...,DTB8,DTB9,DTB10,ILLNESS,MOD,TOD,AAD,SBNND,PLCWGT,TABLE62
0,N10001N,2,0,8,8,2,1,1,0,0,...,0,0,0,0,0,-1,-1,-1,-2,-1
1,N10002P,1,0,2,0,4,4,3,0,0,...,0,0,0,0,0,-1,-1,-1,-2,-1
2,N10003Q,1,0,8,8,2,1,3,0,0,...,0,0,1,3,0,-1,-1,-1,-2,-1
3,N10004R,2,0,8,8,2,1,-8,-8,1,...,1,0,0,-1,0,-1,-1,-1,-2,-1
4,N10005S,2,0,8,8,2,1,3,0,1,...,0,0,0,0,0,-1,-1,-1,-2,-1


In [4]:
#Load and inspect the response data
ncds_response_data = pd.read_csv('datasets/ncds_response.txt', delimiter='\t', low_memory=False)
# Print shapes
print "Shape of data:", ncds_response_data.shape
ncds_response_data.head()

Shape of data: (18558, 18)


,NCDSID,N622,BSTATUS,COBIRTH,MULTIPNO,MULTCODE,ETHNICID,OUTCME00,OUTCME01,OUTCME02,OUTCME03,OUTCME04,OUTCME05,OUTCME06,OUTCMEBM,OUTCME07,OUTCME08,OUTCME09
0,N10001N,2,0,1,-1,-1,1,1,1,1,1,1,1,1,1,1,1,1
1,N10002P,1,0,1,-1,-1,1,1,1,1,1,1,1,1,1,1,1,1
2,N10003Q,1,0,1,-1,-1,1,1,1,1,1,7,7,7,6,7,7,7
3,N10004R,2,0,1,-1,-1,1,1,1,1,1,1,1,1,1,2,2,3
4,N10005S,2,0,2,-1,-1,5,1,1,1,1,2,2,2,6,6,6,6


Let us first join all the threee datasets, so we can work on all the data together.

In [5]:
# Join datasets
ncds_merged_data = pd.merge(left=ncds_data,right=ncds_pms_data,how='left',left_on='ncdsid',right_on='NCDSID')
ncds_merged_data = pd.merge(left=ncds_merged_data,right=ncds_response_data,how='left',left_on='ncdsid',right_on='NCDSID')
print "Shape of data:", ncds_merged_data.shape

Shape of data: (18558, 1837)


In [24]:
#Hide

# Utility methods for data exploring and cleaning
def evaluate_data(df):
    # Check for range of unique values for the train data
    for i in range(df.shape[1]):
        vals = np.unique(df.iloc[:, i])
        if len(vals) < 15:
            print '(Categorical) {} unique values - {}: {}'.format(len(vals), df.columns[i], vals)
        else:
            print '(Continuous) range of values - ', df.columns[i], ': {} to {}'.format(df.iloc[:, i].min(), df.iloc[:, i].max())

def evaluate_epil_columns(df):
    for column in epil_columns:
        vals = np.unique(df[column])
        if len(vals) < 15:
            print '(Categorical) {} unique values - {}: {}'.format(len(vals), column, vals)
        else:
            print '(Continuous) range of values - ', column, ': {} to {}'.format(df[column].min(), df[column].max())


def columns_with_null(df):
    count = 0
    for column in df.columns:
        df_missing = df[df[column].isnull()]
        if df_missing.shape[0] > 0:
            print "Predictor " , column, " contain null values / Count = " ,df_missing.shape[0]
            count = count +1
    print "Total number of columns with null:",count

def find_columns_with_nulls(df):
    columns = []
    for column in df.columns:
        df_missing = df[df[column].isnull()]
        if df_missing.shape[0] > 0:
            columns.append(column)
    
    return columns

def find_columns_without_nulls(df):
    columns = []
    for column in df.columns:
        df_missing = df[df[column].isnull()]
        if df_missing.shape[0] == 0:
            columns.append(column)
    
    return columns

#### Step 2: Understand the data

1. Explore the data
2. Understand the predictors, what they mean in real life
3. Understand the values of each predictors

##### Data Analysis / Exploration

Most of our exploration was done in the data exploration step of our project. The details of the tasks we performed can be viewed [here](20_exploratory_data_analysis_02.ipynb).

#### Step 3: Handle missing data

Are there any missing values, if there are:
1. Can we impute them based on some algorithm
2. Remove or ignore them
3. Assume values based on common sense or prior knowledge

In [19]:
### Utility methods for data scrubbing

# Remove spaces from data
def convert_spaces_to_null(data):
    data = data.replace([' '],[None]) 
    return data

# Fill empty columns using median for the column
def fill_with_median(x_fill):
    x_fill = x_fill.groupby(x_fill.columns, axis = 1).transform(lambda x: x.fillna(x.median()))
    return x_fill

# Fill empty columns using mean for the column
def fill_with_mean(x_fill):
    x_fill = x_fill.groupby(x_fill.columns, axis = 1).transform(lambda x: x.fillna(x.mean()))
    return x_fill

# Fill empty columns using KNN
def fill_with_knn(x_fill,k,features,response):
    model = KNN(n_neighbors=k)
    df_missing = x_fill[x_fill[response].isnull()]
    df_filled = x_fill[~x_fill[response].isnull()]
    df_predict = df_missing.copy()
    try:
        model.fit(df_filled[features],df_filled[response])
        df_predict.loc[:, response] = model.predict(df_missing[features])
        df_final = pd.concat([df_filled, df_predict])
    except ValueError:
        df_predict.loc[:, response] = None
        df_final = pd.concat([df_filled, df_predict])
    return df_final

First let is creata copy of the data and remove duplicate columns that were created when we merged the three datasets. Also from our data exploration step we found that the data with balnsk ave space and we need to convert them to nulls so it is easier to handle null checking in python. Then we need to convert all the column types to float type.

In [12]:
# Make a copy
ncds_data_clean = ncds_merged_data.copy() 

# drop the ID columns
ncds_data_clean =  ncds_data_clean.drop(["ncdsid","NCDSID_x","NCDSID_y"],axis=1)
# Drop other duplicate columns
ncds_data_clean =  ncds_data_clean.drop(["N622_x","BSTATUS_x"],axis=1)


# Convert spaces in the data to nulls
ncds_data_clean = convert_spaces_to_null(ncds_data_clean)

# Convert all columns to float
for column in ncds_data_clean.columns:
    ncds_data_clean[column] = ncds_data_clean[column].astype(float)

The pms file ncds_pms_additionals has 16990 cases only, so we need to impute the columns to fill the null values.

The details on what we chose as the default value for each feature is listed here:

-  **POD**	, Label : Q6:Place of Delivery , Default Value = 5.0	, Label : Elsewhere      
-  **BOOKING**	, Label : Q26b: Booking In place  , Default Value = 3.0	, Label : Other place    
-  **PLANC**	, Label : Q21b: Place of Antenatal care , Default Value = -8.0	, Label : No information 
-  **DIASTOL**	, Label : Q29a: Diastolic Blood Pressure , Default Value = -2.0	, Label : Not applicable 
-  **MAXDBP**	, Label : Q29b: Maximum Diatolic Blood Pressure , Default Value = -8.0	, Label : No information 
-  **ALBECL**	, Label : Q31: Albuminuria and Eclampsia , Default Value = -8.0	, Label : No information 
-  **XRAY**	, Label : Q36: X-Ray given , Default Value = -8.0	, Label : No information 
-  **ABNORM0X**	, Label : Q37: Obstetric, pregnancy abnormality - No information , Default Value = -2.0	, Label : Not applicable 
-  **ABNORM00**	, Label : Q37: No Obstetric, pregnancy abnormality , Default Value = -2.0	, Label : Not applicable 
-  **ABNORM01**	, Label : Q37: Obstetric, pregnancy abnormality - Diabetes , Default Value = -2.0	, Label : Not applicable 
-  **ABNORM02**	, Label : Q37: Obstetric, pregnancy abnormality - Heart , Default Value = -2.0	, Label : Not applicable 
-  **ABNORM03**	, Label : Q37: Obstetric, pregnancy abnormality - Active TB , Default Value = -2.0	, Label : Not applicable 
-  **ABNORM04**	, Label : Q37: Obstetric, pregnancy abnormality - influenza , Default Value = -2.0	, Label : Not applicable 
-  **ABNORM05**	, Label : Q37: Obstetric, pregnancy abnormality - German Measles , Default Value = -2.0	, Label : Not applicable 
-  **ABNORM06**	, Label : Q37: Obstetric, pregnancy abnormality - Disproportion , Default Value = -2.0	, Label : Not applicable 
-  **ABNORM07**	, Label : Q37: Obstetric, pregnancy abnormality - External version , Default Value = -2.0	, Label : Not applicable 
-  **ABNORM08**	, Label : Q37: Obstetric, pregnancy abnormality - Epilepsy , Default Value = -2.0	, Label : Not applicable 
-  **ABNORM09**	, Label : Q37: Obstetric, pregnancy abnormality - Other , Default Value = -2.0	, Label : Not applicable 
-  **BLEED**	, Label : Q37: Bleeding in Pregnancy and before delivery  , Default Value = -1.0	, Label : No information 
-  **AD2HOSP**	, Label : Q38a: Admission to hospital , Default Value = -1.0	, Label : No information 
-  **ADTYPE**	, Label : Q39: Type of Labour or Delivery Admission (Hospital) , Default Value = -1.0	, Label : No information 
-  **PRESENT**	, Label : Q44: Presenting Part , Default Value = -1.0	, Label : No information 
-  **LDRUG00**	, Label : Q49a: No drugs of this type , Default Value = -2.0	, Label : Not applicable 
-  **LDRUG01**	, Label : Q49a: Chloral, Welldorm  , Default Value = -2.0	, Label : Not applicable 
-  **LDRUG02**	, Label : Q49a: Barbiturate , Default Value = -2.0	, Label : Not applicable 
-  **LDRUG03**	, Label : Q49a: Heroin , Default Value = -2.0	, Label : Not applicable 
-  **LDRUG04**	, Label : Q49a: Largactil (chlorpomazine) , Default Value = -2.0	, Label : Not applicable 
-  **LDRUG05**	, Label : Q49a: Sparine (promazine) , Default Value = -2.0	, Label : Not applicable 
-  **LDRUG06**	, Label : Q49a: Phenergan (promethazine) , Default Value = -2.0	, Label : Not applicable 
-  **LDRUG07**	, Label : Q49a: Doriden  , Default Value = -2.0	, Label : Not applicable 
-  **LDRUG08**	, Label : Q49a: Oblivon , Default Value = -2.0	, Label : Not applicable 
-  **LDRUG09**	, Label : Q49a: Other   , Default Value = -2.0	, Label : Not applicable 
-  **ATHETIC**	, Label : Q50: Anaesthetic , Default Value = 9.0	, Label : No information 
-  **RESUS**	, Label : Q55: Resuscitation   , Default Value = -2.0	, Label : Not applicable 
-  **DTB1**	, Label : Q56: Drugs to baby (None)  , Default Value = -2.0	, Label : Not applicable 
-  **DTB2**	, Label : Q56: Drugs to baby (Coranine) , Default Value = -2.0	, Label : Not applicable 
-  **DTB3**	, Label : Q56: Drugs to baby (Lobeline) , Default Value = -2.0	, Label : Not applicable 
-  **DTB4**	, Label : Q56: Drugs to baby (Sedatives) , Default Value = -2.0	, Label : Not applicable 
-  **DTB5**	, Label : Q56: Drugs to baby (Antagonists, nalorphine, levalorfan) , Default Value = -2.0	, Label : Not applicable 
-  **DTB6**	, Label : Q56: Drugs to baby (Synkavit, Vikastab)  , Default Value = -2.0	, Label : Not applicable 
-  **DTB7**	, Label : Q56: Drugs to baby (Sulphonamides)  , Default Value = -2.0	, Label : Not applicable 
-  **DTB8**	, Label : Q56: Drugs to baby (Penicilin) , Default Value = -2.0	, Label : Not applicable 
-  **DTB9**	, Label : Q56: Drugs to baby (Streptomycin)    , Default Value = -2.0	, Label : Not applicable 
-  **DTB10**	, Label : Q56: Drugs to baby (Other antibiotics) , Default Value = -2.0	, Label : Not applicable 
-  **ILLNESS**	, Label : Q59: Baby's Illness  , Default Value = -1.0	, Label : No information 
-  **MOD**	, Label : Q61: Month of Death  , Default Value = 0.0	, Label : Survivor (livebirth /lived)    
-  **TOD**	, Label : Q61: Time of death  , Default Value = -1.0	, Label : Not applicable 
-  **AAD**	, Label : Q61: Age at Death  , Default Value = -1.0	, Label : Not applicable 
-  **SBNND**	, Label : Q61: Still Birth or Neo-natal Death (Dervied)  , Default Value = -1.0	, Label : Survivor       
-  **PLCWGT**	, Label : Placental Weight , Default Value = -2.0	, Label : Not applicable 
-  **TABLE62**	, Label : Time of death for still births and neonatal deaths (Table 62) , Default Value = -1.0	, Label : Not applicable 

In [13]:
### method to fill nulls in the pms dataset
def fill_pms_columns(x_fill):
    for index, row in x_fill.iterrows():
        # Q6:Place of Delivery  
        if pd.isnull(row["POD"]):
            x_fill.set_value(index, 'POD', 5.0)
        # Q26b: Booking In place  
        if pd.isnull(row["BOOKING"]):
            x_fill.set_value(index, 'BOOKING', 3.0)
        # Q21b: Place of Antenatal care  
        if pd.isnull(row["PLANC"]):
            x_fill.set_value(index, 'PLANC', -8.0)
        # Q29a: Diastolic Blood Pressure  
        if pd.isnull(row["DIASTOL"]):
            x_fill.set_value(index, 'DIASTOL', -2.0)
        # Q29b: Maximum Diatolic Blood Pressure 
        if pd.isnull(row["MAXDBP"]):
            x_fill.set_value(index, 'MAXDBP', -2.0)
        # Q31: Albuminuria and Eclampsia
        if pd.isnull(row["ALBECL"]):
            x_fill.set_value(index, 'ALBECL', -8.0)
        # Q36: X-Ray given
        if pd.isnull(row["XRAY"]):
            x_fill.set_value(index, 'XRAY', -8.0)
        # Q37: Obstetric, pregnancy abnormality - No information
        if pd.isnull(row["ABNORM0X"]):
            x_fill.set_value(index, 'ABNORM0X', -2.0)
        # Q37: No Obstetric, pregnancy abnormality
        if pd.isnull(row["ABNORM00"]):
            x_fill.set_value(index, 'ABNORM00', -2.0)
        # Q37: Obstetric, pregnancy abnormality - Diabetes
        if pd.isnull(row["ABNORM01"]):
            x_fill.set_value(index, 'ABNORM01', -2.0)
        # Q37: Obstetric, pregnancy abnormality - Heart 
        if pd.isnull(row["ABNORM02"]):
            x_fill.set_value(index, 'ABNORM02', -2.0)
        # Q37: Obstetric, pregnancy abnormality - Active TB 
        if pd.isnull(row["ABNORM03"]):
            x_fill.set_value(index, 'ABNORM03', -2.0)
        # Q37: Obstetric, pregnancy abnormality - influenza 
        if pd.isnull(row["ABNORM04"]):
            x_fill.set_value(index, 'ABNORM04', -2.0)
        # Q37: Obstetric, pregnancy abnormality - German Measles 
        if pd.isnull(row["ABNORM05"]):
            x_fill.set_value(index, 'ABNORM05', -2.0)
        # Q37: Obstetric, pregnancy abnormality - Disproportion 
        if pd.isnull(row["ABNORM06"]):
            x_fill.set_value(index, 'ABNORM06', -2.0)
        # Q37: Obstetric, pregnancy abnormality - External version 
        if pd.isnull(row["ABNORM07"]):
            x_fill.set_value(index, 'ABNORM07', -2.0)
        # Q37: Obstetric, pregnancy abnormality - Epilepsy 
        if pd.isnull(row["ABNORM08"]):
            x_fill.set_value(index, 'ABNORM08', -2.0)
        # Q37: Obstetric, pregnancy abnormality - Other 
        if pd.isnull(row["ABNORM09"]):
            x_fill.set_value(index, 'ABNORM09', -2.0)
        # Q37: Bleeding in Pregnancy and before delivery 
        if pd.isnull(row["BLEED"]):
            x_fill.set_value(index, 'BLEED', -1.0)
        # Q38a: Admission to hospital  
        if pd.isnull(row["AD2HOSP"]):
            x_fill.set_value(index, 'AD2HOSP', -1.0)
        # Q39: Type of Labour or Delivery Admission (Hospital)  
        if pd.isnull(row["ADTYPE"]):
            x_fill.set_value(index, 'ADTYPE', -1.0)
        # Q44: Presenting Part  
        if pd.isnull(row["PRESENT"]):
            x_fill.set_value(index, 'PRESENT', -1.0)
        # Q49a: No drugs of this type  
        if pd.isnull(row["LDRUG00"]):
            x_fill.set_value(index, 'LDRUG00', -2.0)
        # Q49a: Chloral, Welldorm  
        if pd.isnull(row["LDRUG01"]):
            x_fill.set_value(index, 'LDRUG01', -2.0)
        # Q49a: Barbiturate   
        if pd.isnull(row["LDRUG02"]):
            x_fill.set_value(index, 'LDRUG02', -2.0)
        # Q49a: Heroin   
        if pd.isnull(row["LDRUG03"]):
            x_fill.set_value(index, 'LDRUG03', -2.0)
        # Q49a: Largactil (chlorpomazine)   
        if pd.isnull(row["LDRUG04"]):
            x_fill.set_value(index, 'LDRUG04', -2.0)
        # Q49a: Sparine (promazine)    
        if pd.isnull(row["LDRUG05"]):
            x_fill.set_value(index, 'LDRUG05', -2.0)
        # Q49a: Phenergan (promethazine)    
        if pd.isnull(row["LDRUG06"]):
            x_fill.set_value(index, 'LDRUG06', -2.0)
        # Q49a: Doriden    
        if pd.isnull(row["LDRUG07"]):
            x_fill.set_value(index, 'LDRUG07', -2.0)
        # Q49a: Oblivon    
        if pd.isnull(row["LDRUG08"]):
            x_fill.set_value(index, 'LDRUG08', -2.0)
        # Q49a: Other    
        if pd.isnull(row["LDRUG09"]):
            x_fill.set_value(index, 'LDRUG09', -2.0)
        # Q50: Anaesthetic    
        if pd.isnull(row["ATHETIC"]):
            x_fill.set_value(index, 'ATHETIC', -2.0)
        # Q55: Resuscitation    
        if pd.isnull(row["RESUS"]):
            x_fill.set_value(index, 'RESUS', -2.0)
        # Q56: Drugs to baby (None)      
        if pd.isnull(row["DTB1"]):
            x_fill.set_value(index, 'DTB1', -2.0)
        # Q56: Drugs to baby (Coranine)       
        if pd.isnull(row["DTB2"]):
            x_fill.set_value(index, 'DTB2', -2.0)
        # Q56: Drugs to baby (Lobeline)       
        if pd.isnull(row["DTB3"]):
            x_fill.set_value(index, 'DTB3', -2.0)
        # Q56: Drugs to baby (Sedatives)       
        if pd.isnull(row["DTB4"]):
            x_fill.set_value(index, 'DTB4', -2.0)
        # Q56: Drugs to baby (Antagonists, nalorphine, levalorfan)       
        if pd.isnull(row["DTB5"]):
            x_fill.set_value(index, 'DTB5', -2.0)
        # Q56: Drugs to baby (Synkavit, Vikastab)      
        if pd.isnull(row["DTB6"]):
            x_fill.set_value(index, 'DTB6', -2.0)
        # Q56: Drugs to baby (Sulphonamides)      
        if pd.isnull(row["DTB7"]):
            x_fill.set_value(index, 'DTB7', -2.0)
        # Q56: Drugs to baby (Penicilin)      
        if pd.isnull(row["DTB8"]):
            x_fill.set_value(index, 'DTB8', -2.0)
        # Q56: Drugs to baby (Streptomycin)       
        if pd.isnull(row["DTB9"]):
            x_fill.set_value(index, 'DTB9', -2.0)
        # Q56: Drugs to baby (Other antibiotics)       
        if pd.isnull(row["DTB10"]):
            x_fill.set_value(index, 'DTB10', -2.0)
        # Q59: Baby's Illness       
        if pd.isnull(row["ILLNESS"]):
            x_fill.set_value(index, 'ILLNESS', -1.0)
        # Q61: Month of Death       
        if pd.isnull(row["MOD"]):
            x_fill.set_value(index, 'MOD', 0.0)
        # Q61: Time of death        
        if pd.isnull(row["TOD"]):
            x_fill.set_value(index, 'TOD', -1.0)
        # Q61: Age at Death        
        if pd.isnull(row["AAD"]):
            x_fill.set_value(index, 'AAD', -1.0)
        # Q61: Still Birth or Neo-natal Death (Dervied)         
        if pd.isnull(row["SBNND"]):
            x_fill.set_value(index, 'SBNND', -1.0)
        # Placental Weight         
        if pd.isnull(row["PLCWGT"]):
            x_fill.set_value(index, 'PLCWGT', -2.0)
        # Time of death for still births and neonatal deaths (Table 62)          
        if pd.isnull(row["TABLE62"]):
            x_fill.set_value(index, 'TABLE62', -1.0)
    return x_fill

# Impute missing data from joined columns using default values
ncds_data_clean = fill_pms_columns(ncds_data_clean)

The columns which help us determine if the patient has epilepsy or not we need to make sure while imputing them to set the values to a default value that does not refect epilepsy

In [ ]:
### method to fill nulls in the dataset
def fill_epilepsy_columns(x_fill):
    for index, row in x_fill.iterrows():
        # Q6:Place of Delivery  
        if pd.isnull(row["n2032"]):
            x_fill.set_value(index, 'n2032', -1.0)

# Impute missing data for epilepsy columns
ncds_data_clean = fill_epilepsy_columns(ncds_data_clean)

For the remaining columns for which we have nulls, let us use an algorithim to impute the values

In [20]:
# find the columns without nulls
columns_no_nulls = find_columns_without_nulls(ncds_data_clean)
#print columns_no_null

# Find columns with nulls
columns_with_nulls = find_columns_with_nulls(ncds_data_clean)

# Impute missing data with knn
for column in columns_with_nulls:
    ncds_data_clean = fill_with_knn(ncds_data_clean,15,columns_no_nulls,column)

Let us check if we have imputed all the empty columns

In [25]:
# Get the columns which have still null data
columns_with_null(ncds_data_clean)

Predictor  n339  contain null values / Count =  3134
Predictor  n149  contain null values / Count =  3139
Predictor  n1860  contain null values / Count =  3133
Predictor  n2411  contain null values / Count =  3905
Predictor  n2414  contain null values / Count =  3905
Predictor  n1732  contain null values / Count =  3905
Predictor  n1733  contain null values / Count =  3905
Predictor  n1953  contain null values / Count =  3905
Predictor  n2060  contain null values / Count =  3905
Predictor  n2062  contain null values / Count =  3905
Predictor  n2064  contain null values / Count =  3905
Predictor  n2066  contain null values / Count =  3905
Predictor  n2068  contain null values / Count =  3905
Predictor  n2070  contain null values / Count =  3905
Predictor  n2072  contain null values / Count =  3905
Predictor  n2074  contain null values / Count =  3905
Predictor  n2076  contain null values / Count =  3905
Predictor  n2078  contain null values / Count =  3905
Predictor  n2080  contain null

We can see that we still have columns with null values even after trying to impute with knn. So for the remaining columns we can impute with median values for the columns

In [26]:
# Impute missing data with median values
ncds_data_clean = fill_with_median(ncds_data_clean)

# Get the columns which have still null data
columns_with_null(ncds_data_clean)

Total number of columns with null: 0


We finally have a dataset that is filled with data and has no null values

#### Step 4: Identify Epilepsy Records

In our data a patient is assumed to be epileptic is one or more conditions are satisified in the dataset. We need to check all the conditions in the data and determine if the patient is epileptic.

In [30]:
# Columns the help us identify if the patient has epilepsy
epil_columns = ["n390","n391","n392","n415", "n1842", "n1307", "n1308", "n1309", "n1314", "n1317", "n1477", "n1478", "n1479", "n2416", "n2663", "n2664", "n2665"
                , "n2666", "n2667", "n1893", "n1894", "n1895", "n1904", "n1910", 'n1817', 'n1818',  'n1394', 'n1502', 'n2615', 'n2616']

# Identify if patient has epilepsy
ncds_data_clean["epileptic"] = 0
for index, row in ncds_data_clean.iterrows():
    # 1M Reason for Special Education MC1:3
    if row["n390"] == 10.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 1M Reason for Special Education MC2:3
    if row["n391"] == 10.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 1M Reason for Special Education MC2:3
    if row["n392"] == 10.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 1M Epileptic condition
    if row["n415"] >= 3.0 :
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 12D Epilepsy identification
    if row["n1842"] == 5.0 :
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2P Has child had epilepsy attacks-MC 1:3
    if (row["n1307"] >= 1.0 and row["n1307"] <= 5.0):
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2P Has child had epilepsy attacks-MC 2:3
    if (row["n1308"] >= 1.0 and row["n1308"] <= 5.0):
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2P Has child had epilepsy attacks-MC 3:3
    if (row["n1309"] >= 1.0 and row["n1309"] <= 5.0):
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2P Age at most recent epilepsy attack
    if (row["n1314"] >= 0.0):
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2P Age at 1st epilepsy attack
    if (row["n1317"] >= 0.0):
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2M Reason for special education - MC1:3
    if row["n1477"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2M Reason for special education - MC2:3
    if row["n1478"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2M Reason for special education - MC3:3
    if row["n1479"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3P Type hcap for which will require help
    if row["n2416"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3P Nature of child-s disability-MC 1:5
    if row["n2663"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3P Nature of child-s disability-MC 2:5
    if row["n2664"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3P Nature of child-s disability-MC 3:5
    if row["n2665"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3P Nature of child-s disability-MC 4:5
    if row["n2666"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3P Nature of child-s disability-MC 5:5
    if row["n2667"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3M Category of child's handicap MC1:3
    if row["n1893"] == 8.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3M Category of child's handicap MC2:3
    if row["n1894"] == 8.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3M Category of child's handicap MC3:3
    if row["n1895"] == 8.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3M Reason for hosp admiss last 12 mnths
    if row["n1904"] == 17.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3M Reason hosp outpatient last yr
    if row["n1910"] == 17.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3M Epilepsy
    if row["n2032"] > 1.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    
    # New columns 
    
    # 1D Defects found in NCDS1 sample-MC 1:4
    if row["n1817"] > 0.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 1D Defects found in NCDS1 sample-MC 2:4
    if row["n1818"] > 0.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 1D Defects found in NCDS1 sample-MC 3:4
    if row["n1819"] > 0.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2P Ever seen specialist-convulsions,fits
    if row["n1394"] == 5.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2M Has child ever had convulsions
    if any(row["n1502"] == s for s in [2.0,3.0,4.0]):
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3P When convulsions,fits 1st occured
    if any(row["n2615"] == s for s in [1.0,2.0,3.0,4.0,5.0,6.0]):
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3P Convulsions-most recent occurrence
    if any(row["n2616"] == s for s in [1.0,2.0,3.0,4.0,5.0,6.0,7.0]):
        ncds_data_clean.set_value(index, 'epileptic', 1)
        
print "Number of rows with epilepsy",ncds_data_clean[ncds_data_clean["epileptic"] == 1].shape[0]

Number of rows with epilepsy 1710


In [31]:
# Remove the epilepsy columns from the data
ncds_data_no_indicators=ncds_data_clean.copy()
ncds_data_no_indicators.drop(epil_columns,inplace=True,axis=1)
print "Shape of dataset: " , ncds_data_no_indicators.shape

Shape of dataset:  (18558, 1803)


#### Step 5: Save cleaned data

Save our cleaned dataset so we can use it later in our modeling step

In [34]:
### Write the dataframe to a csv file
ncds_data_no_indicators.to_csv('datasets/ncds_data_no_indicators.csv', sep=',',index=False)